In [ ]:
# -*- coding: utf-8 -*-

"""
30 de junio de 2020

Extracción de Información de un corpus histórico

@author: Antonio Anuncibay Zapata

"""

## Primera parte

Procesamiento inicial

In [1]:
#Importamos las bibliotecas necesarias

import nltk
import nltk.chunk, nltk.tag
from nltk.corpus import cess_esp
from nltk.tokenize import word_tokenize
from nltk import UnigramTagger, BigramTagger, TrigramTagger
from nltk.tag.hmm import HiddenMarkovModelTagger
from nltk.chunk.util import conlltags2tree, tree2conlltags
from nltk import ChunkParserI
from nltk.tree import *
import random


In [2]:
#Cargamos el corpus hipótetico del histórico de pedidos de la compañía 

corpus = ["me puede enviar cuatro bocadillos variados", "quisiera tres perros calientes", "seis pasteles de limón", "Por favor, me puedes enviar una tarta y cinco pastelitos", "para enviar, ocho piezas de pollo variadas", "Me pones un plato de pasta", "siete ensaladas", "Quisiera ordenar cuatro bocadillos con chorizo", "mi orden sería pescado con patatas", "quisiera dos pasteles de nata", "quiero una sopa de cebolla", "dos carnes", "quisiera pedir tres pizzas de hongos y dos shawarma", "me gustaría comer bocadillo de costillas", "por favor, un refresco y cinco patatas fritas", "tengo antojos de un pastel de chocolate", "para comer dos raciones de albondigas", "quiero llevar tres torrijas", "¿Me sirves un pincho de tortilla, por favor?", "quiero una pizza margherita", "por favor, dos hamburguesas", "nueve donus", "podría tener cuatro tortillas", "nueve pasteles", "gazpacho y bocadillos", "6 empanadas", "cuatro raciones de croquetas","una tabla de quesos", "ocho empanadas", "me gustaría comer una ensalada", "pasta con carne", "quisiera ordenar tres chuletas y una ensalada", "cinco patatas fritas", "nueve pescados", "alubias, pan y cuatro bocadillos de jamón", "nueve ensaladas cesar", "dos gazpachos", "paella y pan",  "me gustaría comer bocadillo de costillas", "cinco pollos con patatas" ,"quisiera pedir tres pizzas de queso y dos bocadillos de jamon", "me gustaría comer bocadillo de anchoa con refresco grande", "un agua mineral y cinco croquetas de hongos", "una tortilla y cuatro hamburguesas", "tres pinchos de queso con jamon y dos pizzas", "cinco porciones de pastel", "tres paellas", "siete tortillas y pan con ajo", "arroz a la marinera", "una ración de rabas"]

#len(corpus) #se preparó un corpus con 50 ordenes diferentes

In [3]:
#Cargamos las frases anotadas del Corpus CESS en español

sents = cess_esp.tagged_sents()

#print("sents")   #se puede comprobar las frases cargadas


In [4]:
#El primer paso estaría en decidir con cual de los taggers es el más adecuado  
#para el etiquetado inicial del corpus de trabajo de la empresa, 
#haciendo un entrenamiento del corpus CESS

#Cargamos el conjunto de frases anotadas en dos conjuntos: 
##train: 90% de las frases del conjunto
##test: 10% restante 

sents_train = []
sents_test = []

for i in range(len(sents)):
    if i % 10:
        sents_train.append(sents[i])
    else:
        sents_test.append(sents[i])

In [5]:
#Se establecen los cuatro taggers para conocer su despeño

unigram_tagger = UnigramTagger(sents_train)
bigram_tagger = BigramTagger(sents_train, backoff = unigram_tagger)
trigram_tagger = TrigramTagger(sents_train, backoff = bigram_tagger)
hmm_tagger = HiddenMarkovModelTagger.train(sents_train)

print("  Porcentaje de aciertos")
print(" \n\n *Unigramas: ", round(unigram_tagger.evaluate(sents_test)*100), "%")
print(" \n\n *Bigramas:", round(bigram_tagger.evaluate(sents_test)*100), "%")
print(" \n\n *Trigramas:", round(trigram_tagger.evaluate(sents_test)*100), "%")
print(" \n\n *HMMs:", round(hmm_tagger.evaluate(sents_test)*100), "%")

  Porcentaje de aciertos
 

 *Unigramas:  88 %
 

 *Bigramas: 89 %
 

 *Trigramas: 89 %
 

 *HMMs: 90 %


Conclusión: 

    * El que presenta mejor porcentaje de aciertos es el modelo de Hidden Markov; por tanto, se decide elegir este 
    para realizar el proceso de etiquetado del corpus histórico de trabajo de la empresa.

## Segunda Parte

Preparación del cuerpo de trabajo

* Para realizar el proceso de Extracción de Información primero se debe realizar el etiquetado
del corpus histórico de la empresa, para luego aplicarle el proceso de RegexParser con la información a extraer. 

In [6]:
#Entrenamos el corpus entero de frases anotadas CESS en el modelo de Hidden Markov

Esp_POS_tagger = nltk.HiddenMarkovModelTagger.train(sents)


In [7]:
## Se crea una gramática para extraer la información según la información morfológica 
#del modelo de etiquetado EAGLE



grammar = r"""

   comida:{<ncfs000|ncms000|ncmp000|rg>?<sps00>*<ncfs000|ncfp000|ncms000|ncmp000|da0fs0>}  ##patrón para frases compuestas, como
                                                                                            #bocadillo con chorizo
          {<ncfs000|ncms000|ncfp000>?<aq0fs0|aq0fp0|da0fs0|ncms000>}  #patrón para frases compuestas como patatas fritas
          {<ncms000>}                                                #patrón para definir nombres o sustantivos
          {<ncfs000>}
          {<ncfp000>}
          {<ncmp000>}
          {<da0fs0>}
          {<ncfp000>}
          {<rg>}

   cantidad:{<di0ms0>}  ##patrón para definir las cantidades, el POS etiqueta en estas frases u oraciones
                           #las cantidades como determinantes y no como numerales, de igual manera
                           #se incluye la etiqueta EAGLE para ambos
            {<di0fs0>}
            {<dn0cp0>}
            {<MCMP00>}
            {<MCFS00>}
            {<MCFP00>}
            {<MCFS00>}
            {<MCCP00>}

"""

rp = nltk.RegexpParser(grammar)

In [8]:
#se declara una función para convertir las cantidades 
#en números aunque según el enunciado del ejercicio no se consideraba necesario. 

def clasificar_cantidad(cantidad):

    
    if (cantidad == "un" or cantidad == "una" or cantidad == 1) : 
        Cant_num = 1
        return(Cant_num)
    if (cantidad == "dos" or cantidad == 2): 
        Cant_num = 2
        return(Cant_num)
    if (cantidad == "tres" or cantidad == 3): 
        Cant_num = 3
        return(Cant_num)
    if (cantidad == "cuatro" or cantidad == 4): 
        Cant_num = 4
        return(Cant_num)
    if (cantidad == "cinco" or cantidad == 5): 
        Cant_num = 5
        return(Cant_num)
    if (cantidad == "seis" or cantidad == 6): 
        Cant_num = 6
        return(Cant_num)
    if (cantidad == "siete" or cantidad == 7): 
        Cant_num = 7
        return(Cant_num)
    if (cantidad == "ocho" or cantidad == 8): 
        Cant_num = 8
        return(Cant_num)
    if (cantidad == "nueve" or cantidad == 9): 
        Cant_num = 9
        return(Cant_num)
    if (cantidad == "diez" or cantidad == 10): 
        Cant_num = 10
        return(Cant_num)


In [9]:
##Función de separación de "cantidades" y "comidas" de
#del resultado del proceso de los diferentes procesos de extracción

def clasificar_rp(tree):
    
    #switches booleanos para la separación de los pedidos sin cantidad (null) a los pedidos con cantidad
    cant_1 = False  
    cant_2 = False
    cant_3 = False
    
    for subtree in tree.subtrees(): #extracción de la información enviada del RegexParser en forma de rama/árbol
            
        if subtree.label() == "comida":          
            cant_1 = True 
            comida = subtree.leaves()
            lista_comida = []
            for a,b in comida:               
                lista_comida.append(a)           
            print("\n\n Comida:", " ".join(lista_comida)) 

        if subtree.label() == "cantidad":
            cant_2 = True
            lista_cantidad = []
            cantidad = subtree.leaves()
            for a,b in cantidad:
                lista_cantidad.append(a)               
            cantidad = " ".join(lista_cantidad)            
            num = clasificar_cantidad(cantidad)           
            print("\n\n Cantidad(es):", num)
        
        if (cant_1 == True and cant_2 == False and cant_3 == False):     
            cant_3 == True          
            print("Cantidad(es): 1")
                         
    print("\n\n ... ")

In [10]:
#Extracción de información basado en RegexParser

def __init__():

    random.shuffle(corpus) # a manera de ejemplo se hace una reorganización al azar del cuerpo de trabajo
                           #para imprimir de manera aleatoria los primeros cinco llamados de la extracción del cuerpo y no las 50 ordenes 
    
    j = 0
    
    for sent in corpus[:2]:
        
        j = j + 1
        print("\n\n Frase #", j)
       
        print("\n\n ***Orden:", sent)
        
        sent_tokens = nltk.word_tokenize(sent) #tokenización del texto
        print("\n\n a. Texto tokenizado:")
        print(sent_tokens)

        sent_tagged = Esp_POS_tagger.tag(sent_tokens) #proceso de etiquetado
        print("\n\n b. Texto etiquetado:")
        print(sent_tagged)

        sent_parser = rp.parse(sent_tagged) #aplicación del análisis según el "grammar" creado de patrones
        print("\n\n  c. Extracción de información con RegexParser:")
        print("c.1. Resultado RegexParser:")
        print(sent_parser)
        clasf = clasificar_rp(sent_parser) #se envía a la función de clasficación
 
__init__()



 Frase # 1


 ***Orden: mi orden sería pescado con patatas


 a. Texto tokenizado:
['mi', 'orden', 'sería', 'pescado', 'con', 'patatas']


 b. Texto etiquetado:
[('mi', 'dp1css'), ('orden', 'ncms000'), ('sería', 'spcms'), ('pescado', 'ncms000'), ('con', 'sps00'), ('patatas', 'ncfp000')]


  c. Extracción de información con RegexParser:
c.1. Resultado RegexParser:
(S
  mi/dp1css
  (comida orden/ncms000)
  sería/spcms
  (comida pescado/ncms000 con/sps00 patatas/ncfp000))


 Comida: orden
Cantidad(es): 1


 Comida: pescado con patatas
Cantidad(es): 1


 ... 


 Frase # 2


 ***Orden: quisiera pedir tres pizzas de queso y dos bocadillos de jamon


 a. Texto tokenizado:
['quisiera', 'pedir', 'tres', 'pizzas', 'de', 'queso', 'y', 'dos', 'bocadillos', 'de', 'jamon']


 b. Texto etiquetado:
[('quisiera', 'sps00'), ('pedir', 'vmn0000'), ('tres', 'dn0cp0'), ('pizzas', 'ncmp000'), ('de', 'sps00'), ('queso', 'np0000l'), ('y', 'cc'), ('dos', 'dn0cp0'), ('bocadillos', 'ncmp000'), ('de', 'sps00'), 

Conclusión: 
     * Una vez que se comprueba el funcionamiento del RegexParser para la extracción de información
     se usa para la preparación del corpus con etiquetado IOB para trabajar con 
     los modelos Unigram, Bigram y Naive Bayes
     

## Segunda Parte

Extracción de Información del corpus con los modelos Unigram, Bigram y Naive Bayes basados en el etiquetado IOB

1. Preparación del corpus histórico con etiquetas IOB basados en el RegexParser

In [11]:
#Preparación del corpus histórico con etiquetado IOB
corpus_IOB = []
for sentence in corpus:
    sentence_tokens = nltk.word_tokenize(sentence)
    sentence_tagged = Esp_POS_tagger.tag(sentence_tokens)
    sentence_parser = rp.parse(sentence_tagged)
    chunked_IOB = tree2conlltags(sentence_parser)
    corpus_IOB.append(conlltags2tree(chunked_IOB))

#Impresión de muestra de etiquetado IOB
#random.shuffle(corpus_IOB)
#print("\n\n", corpus_IOB[:5]) 

#Corpus de prueba para la comprobaciónn del funcionamiento de los otros parser.
corpus_parser = []
for sent in corpus:
    sent_tokens = nltk.word_tokenize(sent)
    sent_tagged = Esp_POS_tagger.tag(sent_tokens)
    corpus_parser.append(sent_tagged)

#Impresión de muestra
#random.shuffle(corpus_parser)
#print("\n\n", corpus_parser[:5])


In [12]:
#Cargamos el Corpus IOB etiquetado en dos conjuntos: 
##train: 90% de las frases del conjunto
##test: 10% restante 

train_sents = []
test_sents = []

for i in range(len(corpus_IOB)):
    if i % 10:
        train_sents.append(corpus_IOB[i])
    else:
        test_sents.append(corpus_IOB[i])


* Anotación: los tres modelos a ser usados trabajan de manera bastante parecida. 
Por un lado tienen una función en la cual separan el etiquetado IOB del corpus presentado (entrenamiento), para luego entrenar el modelo respectivo y volver a construir la frase. Así al presentarse el corpus de prueba usa el entrenamiento del modelo para comprobar su desempeño. 

2. Extracción de Información con UnigramParser

In [13]:
class Unigram_TAG(nltk.ChunkParserI):
    
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] 
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                    in zip(sentence, chunktags)]
        return(nltk.chunk.conlltags2tree(conlltags))

In [14]:
UTagger = Unigram_TAG(train_sents)
print("\n\n", UTagger.evaluate(test_sents))

random.shuffle(corpus_parser)
for sent in corpus_parser[:2]:
    print("\n\n Texto de prueba:")
    print("*Anotado:", sent)
    clasf = UTagger.parse(sent)
    clasificar_rp(clasf)



 ChunkParse score:
    IOB Accuracy:  91.7%%
    Precision:     75.0%%
    Recall:        90.0%%
    F-Measure:     81.8%%


 Texto de prueba:
*Anotado: [('nueve', 'dn0cp0'), ('pasteles', 'ncmp000')]


 Cantidad(es): 9


 Comida: pasteles


 ... 


 Texto de prueba:
*Anotado: [('Quisiera', 'da0mp0'), ('ordenar', 'ao0mp0'), ('cuatro', 'dn0cp0'), ('bocadillos', 'ncmp000'), ('con', 'sps00'), ('chorizo', 'da0fs0')]


 Cantidad(es): 4


 Comida: bocadillos con chorizo


 ... 


3. Extracción de Información con Bigram Parser

In [15]:
class BigramTagger(nltk.ChunkParserI):
    
    def __init__(self, train_sents):
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)] for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data)

    def parse(self, sentence):
        pos_tags = [pos for (words, pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                    in zip(sentence, chunktags)] 
        return nltk.chunk.conlltags2tree(conlltags)

In [25]:
Btagger = BigramTagger(train_sents)
print("\n\n", Btagger.evaluate(test_sents))

random.shuffle(corpus_parser)
for sent in corpus_parser[:2]:
    print("\n\n Texto de prueba:")
    print("*Anotado:", sent)
    clasf = Btagger.parse(sent)
    clasificar_rp(clasf)



 ChunkParse score:
    IOB Accuracy:  83.3%%
    Precision:    100.0%%
    Recall:        80.0%%
    F-Measure:     88.9%%


 Texto de prueba:
*Anotado: [('nueve', 'dn0cp0'), ('pasteles', 'ncmp000')]


 Cantidad(es): 9


 Comida: pasteles


 ... 


 Texto de prueba:
*Anotado: [('tres', 'dn0cp0'), ('pinchos', 'ncmp000'), ('de', 'sps00'), ('queso', 'vmn0000'), ('con', 'sps00'), ('jamon', 'np0000l'), ('y', 'cc'), ('dos', 'dn0cp0'), ('pizzas', 'ncmp000')]


 Cantidad(es): 3


 Comida: pinchos de


 ... 


4. Extracción de Información con Naive Bayes 

In [17]:
class NBTagger(nltk.TaggerI):
    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):        
                featureset = npchunk_features(untagged_sent, i, history)
                train_set.append( (featureset, tag) )
                history.append(tag)       
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):       
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)            
        return zip(sentence, history)

class NBChunker(nltk.ChunkParserI):
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = NBTagger(tagged_sents)

    def parse(self, sentence):       
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)
    




In [24]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    return {"pos": pos, "word" : word, "prevpos": prevpos}

NB_Model = NBChunker(train_sents)
print(NB_Model.evaluate(test_sents))

random.shuffle(corpus_parser)
for sent in corpus_parser[:2]:
    print("\n\n Texto de prueba:")
    print("*Anotado:", sent)
    clasf = NB_Model.parse(sent)
    clasificar_rp(clasf)

ChunkParse score:
    IOB Accuracy:  79.2%%
    Precision:     60.0%%
    Recall:        60.0%%
    F-Measure:     60.0%%


 Texto de prueba:
*Anotado: [('seis', 'dn0cp0'), ('pasteles', 'ncmp000'), ('de', 'sps00'), ('limón', 'da0fs0')]


 Cantidad(es): 6


 Comida: pasteles de limón


 ... 


 Texto de prueba:
*Anotado: [('nueve', 'dn0cp0'), ('pescados', 'ncmp000')]


 Cantidad(es): 9


 Comida: pescados


 ... 


5. Conclusión: 
    * Aunque el Unigram y el Naive tienen un acierto mayor, el Naive tiene una precisión y Recall de casi un 12% mayor en promedio; viéndose reflejada en "comida" con frases compuestas como por ejemplo: plato de pasta, bocadillo de chorizo.
    * Igualmente, el Bigram al tomar en cuenta dos palabras para su extración o clasificación tiene una precisión mucho mayor que el Unigram o Naive, aunque su cantidad de aciertos se vea con un porcentaje menor.



## Parte final: Procesamiento de orden

Función de prueba en donde el usuario puede ingresar un pedido y chequear el resultado según 
los distintos modelos de extracción

In [19]:
def procesar_orden(orden):
    
    print("\n\n Texto a ser procesado:", orden)
    sent_tokens = nltk.word_tokenize(orden)
    sent_tagged = Esp_POS_tagger.tag(sent_tokens)

    print("\n\n Clasificación de la orden:")
    
    print("\n\n 1. RegexParser:")
    sent_parser = rp.parse(sent_tagged)
    clasificar_rp(sent_parser)
    
    ##unigram
    print("\n\n 2. Unigram:")
    clasf_unigram = UTagger.parse(sent_tagged)
    clasificar_rp(clasf_unigram)

    ##bigram
    print("\n\n 3. Bigram:")
    clasf_bigram = Btagger.parse(sent_tagged)
    clasificar_rp(clasf_bigram)

    ##NaiveBayes
    print("\n\n 4. Naives Bayes:")
    clasf_NB = NB_Model.parse(sent_tagged)
    clasificar_rp(clasf_NB)
    
    return()

In [22]:
def pedir_comida(): 

    orden_raw = input() #pedir al usuario una orden de comida
    orden_tag = procesar_orden(orden_raw)
    
pedir_comida()

Quisiera pedir dos bocadillos de jamon, tres tortillas y seis cervezas


 Texto a ser procesado: Quisiera pedir dos bocadillos de jamon, tres tortillas y seis cervezas


 Clasificación de la orden:


 1. RegexParser:


 Cantidad(es): 2


 Comida: bocadillos


 Cantidad(es): 3


 Comida: tortillas


 Cantidad(es): 6


 Comida: cervezas


 ... 


 2. Unigram:


 Comida: Quisiera
Cantidad(es): 1


 Cantidad(es): 2


 Comida: bocadillos de


 Cantidad(es): 3


 Comida: tortillas


 Cantidad(es): 6


 Comida: cervezas


 ... 


 3. Bigram:


 Cantidad(es): 2


 Comida: bocadillos de


 ... 


 4. Naives Bayes:


 Cantidad(es): 2


 Comida: bocadillos de jamon


 Cantidad(es): 3


 Comida: tortillas


 Cantidad(es): 6


 Comida: cervezas


 ... 
